In [32]:
import torch
import torch.nn.functional as F
from transformers import LlamaTokenizer, LlamaForCausalLM
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm # For progress bars
import os

In [33]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
MODEL_PATH = "meta-llama/Llama-2-7b-chat-hf"
print(f"Using device: {DEVICE}")

Using device: cpu


In [34]:
tokenizer = LlamaTokenizer.from_pretrained(MODEL_PATH)
tokenizer.pad_token = tokenizer.eos_token

model = LlamaForCausalLM.from_pretrained(
    MODEL_PATH,
    torch_dtype=torch.float16,
    device_map="auto"
)
model.eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((4096,), eps=1e-05)
    (rotary_e

In [ ]:
def get_original_activations_and_logits(model, inputs):
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)
    
    original_hidden_states = outputs.hidden_states
    
    return outputs.logits, original_hidden_states

def create_malicious_output(tokenizer, original_logits):
    # the very last token in the sequence
    last_token_logits = original_logits[0, -1, :].clone()
    
    # Find index of highest logit
    correct_token_idx = torch.argmax(last_token_logits).item()
    
    # a very unlikely token
    incorrect_token_idx = torch.argmin(last_token_logits).item()
    
    print("--- Logit Swap Attack ---")
    print(f"Original top prediction: '{tokenizer.decode(correct_token_idx)}' (ID: {correct_token_idx})")
    print(f"Target swap token:     '{tokenizer.decode(incorrect_token_idx)}' (ID: {incorrect_token_idx})")
    
    # malicious target by swapping the values
    malicious_target_logits = last_token_logits.clone()
    correct_value = malicious_target_logits[correct_token_idx]
    incorrect_value = malicious_target_logits[incorrect_token_idx]
    
    malicious_target_logits[correct_token_idx] = incorrect_value
    malicious_target_logits[incorrect_token_idx] = correct_value
    
    print(f"New top prediction after swap: '{tokenizer.decode(torch.argmax(malicious_target_logits))}'\n")
    
    return malicious_target_logits.detach()

def reconstruct_internal_state(model, 
                               original_hidden_states, 
                               malicious_target_logits,
                               inputs,
                                 epoch=200, lr=0.01):
    # first hidden state is the input embedding
    input_embeddings = original_hidden_states[0].detach()
    
    reconstructed_states = []
    for i in range(1, len(original_hidden_states)):
        # only need to reconstruct states for last token position
        state = original_hidden_states[i][0, -1, :].clone().detach().to(DEVICE)
        reconstructed_states.append(state.requires_grad_(True))

    # setup the optimizer to update our list of trainable state tensors
    optimizer = torch.optim.Adam(reconstructed_states, lr=lr)
    loss_function = torch.nn.MSELoss()
    
    print(f"--- Reconstructing Internal State (Optimizing {len(reconstructed_states)} tensors) ---")
    
    for step in tqdm(range(epoch), desc="Optimization Progress"):
        optimizer.zero_grad()
        
        total_consistency_loss = 0.0
        
        # start with the fixed embedding of the second to last token
        current_hidden_state = original_hidden_states[0][0, -1, :].unsqueeze(0).unsqueeze(0).detach()
        
        # run the forward pass layer by layer
        for i, layer in enumerate(model.model.layers):
            # simplifying by using the previous layers output as input
            layer_output = layer(current_hidden_state, 
                                 position_ids=position_ids,
                                 attention_mask=attention_mask,
                                 use_cache=False)[0]
            
            # trainable guess for the input of layer `i+1`.
            consistency_loss = loss_function(layer_output.squeeze(), reconstructed_states[i])
            total_consistency_loss += consistency_loss
            
            # output of this layer becomes the input for the next
            current_hidden_state = layer_output

        # final hidden state after the last layer
        final_hidden_state = current_hidden_state
        
        final_hidden_state = model.model.norm(final_hidden_state)
        reconstructed_logits = model.lm_head(final_hidden_state).squeeze()
        
        # penalize the difference between our result and the malicious target
        target_loss = loss_function(reconstructed_logits, malicious_target_logits)
        
        total_loss = target_loss + total_consistency_loss
        
        total_loss.backward()
        optimizer.step()
        
        if step % 50 == 0:
            print(f"Step {step}: Total Loss={total_loss.item():.4f}, Target Loss={target_loss.item():.4f}, Consistency Loss={total_consistency_loss.item():.4f}")

    return [state.detach() for state in reconstructed_states]

In [36]:
def analyze_reconstruction(original_hidden_states, reconstructed_states):
    analysis_results = []
    
    # Compare the last token's hidden state across all layers
    for i in range(len(reconstructed_states)):
        original = original_hidden_states[i+1][0, -1, :].to(DEVICE)
        reconstructed = reconstructed_states[i]
        
        # 1. Calculate overall MSE for the layer
        mse = F.mse_loss(original, reconstructed).item()
        
        # 2. Find the neuron with the minimum absolute difference
        abs_diff = torch.abs(original - reconstructed)
        min_diff_val, min_diff_idx = torch.min(abs_diff, dim=0)
        
        # 3. Store detailed results for this layer
        analysis_results.append({
            'layer_index': i,
            'mse_error': mse,
            'min_abs_difference': min_diff_val.item(),
            'min_diff_neuron_index': min_diff_idx.item()
        })
        
    print("\n--- Analysis of Reconstruction ---")
    mean_mse = np.mean([res['mse_error'] for res in analysis_results])
    print(f"Mean Squared Error across all layers: {mean_mse:.6f}")
    
    best_layer = min(analysis_results, key=lambda x: x['min_abs_difference'])
    print(f"Most plausible neuron found in Layer {best_layer['layer_index']}:")
    print(f"  - Neuron Index: {best_layer['min_diff_neuron_index']}")
    print(f"  - Absolute Difference: {best_layer['min_abs_difference']:.8f}")

    mse_values = [res['mse_error'] for res in analysis_results]
    plt.figure(figsize=(12, 6))
    plt.plot(mse_values, marker='o', linestyle='-')
    plt.title("Reconstruction Error (MSE) vs. Layer Depth")
    plt.xlabel("Decoder Layer Index")
    plt.ylabel("Mean Squared Error")
    plt.grid(True)
    save_path = 'attack_success.pdf'
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()
    
    return analysis_results

In [41]:
# %%
#PROMPT = "The capital of Turkey is"
#PROMPT = "My favorite color is"
PROMPT = "Facebook is social"
inputs = tokenizer(PROMPT, return_tensors="pt").to(DEVICE)

sequence_length = inputs['input_ids'].shape[1]

#STEP 1
original_logits, original_hidden_states = get_original_activations_and_logits(model, inputs)
malicious_target = create_malicious_output(tokenizer, original_logits)

--- Logit Swap Attack ---
Original top prediction: 'networking' (ID: 28127)
Target swap token:     'lichkeit' (ID: 23143)
New top prediction after swap: 'media'



In [42]:


#STEP 2
reconstructed_hidden_states = reconstruct_internal_state(
    model,
    original_hidden_states,
    malicious_target,
    inputs,
    epoch=250,
    lr=0.05
)

#STEP 3
analysis_data = analyze_reconstruction(original_hidden_states, reconstructed_hidden_states)


TypeError: reconstruct_internal_state() missing 1 required positional argument: 'attention_mask'

In [ ]:
#STEP 4: Save the analysis to a CSV file
filename=f"analysis_{PROMPT.replace(' ', '_')}.csv"
df = pd.DataFrame(analysis_data)
df.to_csv(filename, index=False)

In [38]:
current_hidden_state

tensor([[-0.0079,  0.0270, -0.0012,  ...,  0.0032,  0.0051, -0.0337]],
       dtype=torch.float16, grad_fn=<SelectBackward0>)

In [ ]:
seq_length = inputs['input_ids'].shape[1]

In [ ]:
epoch=250
lr=0.05
# first hidden state is the input embedding
input_embeddings = original_hidden_states[0].detach()

reconstructed_states = []
for i in range(1, len(original_hidden_states)):
    # only need to reconstruct states for last token position
    state = original_hidden_states[i][0, -1, :].clone().detach().to(DEVICE)
    reconstructed_states.append(state.requires_grad_(True))

# setup the optimizer to update our list of trainable state tensors
optimizer = torch.optim.Adam(reconstructed_states, lr=lr)
loss_function = torch.nn.MSELoss()

print(f"--- Reconstructing Internal State (Optimizing {len(reconstructed_states)} tensors) ---")

for step in tqdm(range(epoch), desc="Optimization Progress"):
    optimizer.zero_grad()
    
    total_consistency_loss = 0.0
    
    # start with the fixed embedding of the second to last token
    current_hidden_state = original_hidden_states[0][0, -1, :].unsqueeze(0).unsqueeze(0).detach()
    
    # run the forward pass layer by layer
    for i, layer in enumerate(model.model.layers):
        # simplifying by using the previous layers output as input
        layer_output = layer(
                current_hidden_state,
                attention_mask=inputs['attention_mask'],
                position_ids=torch.arange(seq_length, device=DEVICE).unsqueeze(0),
                use_cache=False
            )[0]
        
        # trainable guess for the input of layer `i+1`.
        consistency_loss = loss_function(layer_output.squeeze(), reconstructed_states[i])
        total_consistency_loss += consistency_loss
        
        # output of this layer becomes the input for the next
        current_hidden_state = layer_output

    # final hidden state after the last layer
    final_hidden_state = current_hidden_state
    
    final_hidden_state = model.model.norm(final_hidden_state)
    reconstructed_logits = model.lm_head(final_hidden_state).squeeze()
    
    # penalize the difference between our result and the malicious target
    target_loss = loss_function(reconstructed_logits, malicious_target_logits)
    
    total_loss = target_loss + total_consistency_loss
    
    total_loss.backward()
    optimizer.step()
    
    if step % 50 == 0:
        print(f"Step {step}: Total Loss={total_loss.item():.4f}, Target Loss={target_loss.item():.4f}, Consistency Loss={total_consistency_loss.item():.4f}")

--- Reconstructing Internal State (Optimizing 32 tensors) ---


Optimization Progress:   0%|          | 0/250 [00:00<?, ?it/s]


RuntimeError: The size of tensor a (32) must match the size of tensor b (128) at non-singleton dimension 3

In [45]:
def reconstruct_internal_state(model, 
                               original_hidden_states, 
                               malicious_target_logits,
                               inputs,
                               epoch=200, lr=0.01):
    # Get the sequence length
    seq_length = inputs['input_ids'].shape[1]
    
    # Initialize reconstructed states for each layer (except the first embedding layer)
    reconstructed_states = []
    for i in range(1, len(original_hidden_states)):
        # Only reconstruct the last token's hidden state for each layer
        state = original_hidden_states[i][0, -1, :].clone().detach().to(DEVICE)
        reconstructed_states.append(state.requires_grad_(True))

    # Setup optimizer
    optimizer = torch.optim.Adam(reconstructed_states, lr=lr)
    loss_function = torch.nn.MSELoss()
    
    print(f"--- Reconstructing Internal State (Optimizing {len(reconstructed_states)} tensors) ---")
    
    for step in tqdm(range(epoch), desc="Optimization Progress"):
        optimizer.zero_grad()
        
        total_consistency_loss = 0.0
        
        # Create a custom forward pass that uses our reconstructed states
        # Start with the original embeddings
        current_hidden_states = original_hidden_states[0].clone()
        
        # Process through each layer
        for i, layer in enumerate(model.model.layers):
            # Get the original layer output for comparison
            with torch.no_grad():
                original_layer_output = layer(
                    current_hidden_states,
                    attention_mask=inputs['attention_mask'],
                    position_ids=torch.arange(seq_length, device=DEVICE).unsqueeze(0),
                    use_cache=False
                )[0]
            
            # Calculate consistency loss between original output and our reconstruction
            # for the last token position
            consistency_loss = loss_function(
                original_layer_output[0, -1, :], 
                reconstructed_states[i]
            )
            total_consistency_loss += consistency_loss
            
            # For the next layer's input, use the original hidden states but replace
            # the last token with our reconstructed version
            current_hidden_states = original_layer_output.clone()
            current_hidden_states[0, -1, :] = reconstructed_states[i]

        # Apply final layer norm and compute logits
        final_hidden_state = model.model.norm(current_hidden_states)
        reconstructed_logits = model.lm_head(final_hidden_state[0, -1, :])
        
        # Target loss: difference between reconstructed logits and malicious target
        target_loss = loss_function(reconstructed_logits, malicious_target_logits)
        
        total_loss = target_loss + total_consistency_loss
        
        total_loss.backward()
        optimizer.step()
        
        if step % 50 == 0:
            print(f"Step {step}: Total Loss={total_loss.item():.4f}, Target Loss={target_loss.item():.4f}, Consistency Loss={total_consistency_loss.item():.4f}")

    return [state.detach() for state in reconstructed_states]
def reconstruct_internal_state_v2(model, 
                                  original_hidden_states, 
                                  malicious_target_logits,
                                  inputs,
                                  epoch=200, lr=0.01):
    
    # Initialize reconstructed states for the last token of each layer
    reconstructed_states = []
    for i in range(1, len(original_hidden_states)):
        state = original_hidden_states[i][0, -1, :].clone().detach().to(DEVICE)
        reconstructed_states.append(state.requires_grad_(True))

    optimizer = torch.optim.Adam(reconstructed_states, lr=lr)
    loss_function = torch.nn.MSELoss()
    
    print(f"--- Reconstructing Internal State (Optimizing {len(reconstructed_states)} tensors) ---")
    
    for step in tqdm(range(epoch), desc="Optimization Progress"):
        optimizer.zero_grad()
        
        # Hook to replace hidden states with our reconstructed versions
        def create_hook(layer_idx, reconstructed_state):
            def hook(module, input, output):
                # Replace the last token's hidden state with our reconstruction
                modified_output = output[0].clone()
                modified_output[0, -1, :] = reconstructed_state
                return (modified_output,) + output[1:]
            return hook
        
        # Register hooks for each layer
        hooks = []
        for i, layer in enumerate(model.model.layers):
            if i < len(reconstructed_states):
                hook = layer.register_forward_hook(create_hook(i, reconstructed_states[i]))
                hooks.append(hook)
        
        try:
            # Forward pass with modified hidden states
            with torch.enable_grad():
                outputs = model(**inputs, output_hidden_states=True)
                reconstructed_logits = outputs.logits[0, -1, :]
                
                # Target loss
                target_loss = loss_function(reconstructed_logits, malicious_target_logits)
                
                # Consistency loss
                total_consistency_loss = 0.0
                for i, reconstructed_state in enumerate(reconstructed_states):
                    if i + 1 < len(outputs.hidden_states):
                        original_state = original_hidden_states[i + 1][0, -1, :]
                        consistency_loss = loss_function(original_state, reconstructed_state)
                        total_consistency_loss += consistency_loss
                
                total_loss = target_loss + total_consistency_loss
                
        finally:
            # Remove hooks
            for hook in hooks:
                hook.remove()
        
        total_loss.backward()
        optimizer.step()
        
        if step % 50 == 0:
            print(f"Step {step}: Total Loss={total_loss.item():.4f}, Target Loss={target_loss.item():.4f}, Consistency Loss={total_consistency_loss.item():.4f}")

    return [state.detach() for state in reconstructed_states]


In [ ]:
# %%
PROMPT = "The capital of Turkey is"
PROMPT = "My favorite color is"
inputs = tokenizer(PROMPT, return_tensors="pt").to(DEVICE)

# Remove the manual position_ids and attention_mask creation
# sequence_length = inputs['input_ids'].shape[1]
# last_token_position = sequence_length - 1
# position_ids = torch.tensor([[last_token_position]], device=DEVICE, dtype=torch.long)
# attention_mask = torch.ones((1, 1), dtype=torch.long, device=DEVICE)

#STEP 1
original_logits, original_hidden_states = get_original_activations_and_logits(model, inputs)
malicious_target = create_malicious_output(tokenizer, original_logits)

#STEP 2
reconstructed_hidden_states = reconstruct_internal_state(
    model,
    original_hidden_states,
    malicious_target,
    inputs,  # Pass the full inputs
    epoch=250,
    lr=0.05
)

#STEP 3
analysis_data = analyze_reconstruction(original_hidden_states, reconstructed_hidden_states)


In [47]:
#STEP 2
reconstructed_hidden_states = reconstruct_internal_state_v2(  # or reconstruct_internal_state_v2
    model,
    original_hidden_states,
    malicious_target,
    inputs,
    epoch=250,
    lr=0.05
)


--- Reconstructing Internal State (Optimizing 32 tensors) ---


Optimization Progress:   0%|          | 0/250 [00:00<?, ?it/s]


IndexError: too many indices for tensor of dimension 2